# HW 2: Corporate Bond Pricing (due by 9.21 Fri)

We are going to compute the price of a corporate bond (subject to default) with Monte-Carlo simulation. Assume that 
* the default time of a company follows the exponential distribution with intensity $\lambda=$__`def_rate`__. 
* the riskfree interest rate is $r_f=$__`rf_rate`__ and the maturity of the bond is $T=$__`mat`__. 
* in the case of default, you can recover some portion ($R=$__`recovery_rate`__) of the face value.
* the coupon is 0%, i.e., it is a zero-coupon bond.
* the face value of the bond is 1.0
* use compound rate for discounting; the price of the default-free bond is $e^{-r_f T}$

The Problem 1 of the [2017 ASP Midterm Exam](../files/ASP2017_Midterm.pdf) will be helpful.

### Instruction to upload your HW
* Create a repository named __`PHBS_ASP_2018`__ (and clone it to your PC)
* Copy this file to __`PHBS_ASP_2018/HW2/HW2.ipynb`__  (Please use the same name for repository and ipynb file)
* Adding more code.
* Run your your code to make sure that there's no error.
* Upload (commit and sync) your file.

### 1. First, let's create a pricing function and check the std 

In [1]:
import numpy as np

In [82]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

In [9]:
# First generate exponential random numbers
# Although you can generate directly using fault_time = np.random.exponential(scale=), let's use uniform random numbers.
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(U)

# You can check if the RNs are correct by comparing the means
(default_time.mean(), 1/def_rate)

(10.06426459896198, 10.0)

In [82]:
# Put your code here to price the corporate bond

def corp_bond(mat=10, def_rate=0.1, rf_rate=0.03, recovery=0.3, n_sample=1000):
    U = np.random.uniform(size=n_sample)
    default_time =-(1/def_rate)*np.log(U)
    price=np.where(default_time>mat,np.exp(-rf_rate*mat),recovery*np.exp(-rf_rate*default_time))

    return np.mean(price)
    

# Call your function
corp_bond()

0.44871921351009125

In [55]:
# Find the mean and std by calling the function 100 times.

array=[corp_bond() for k in range(100)]
[np.mean(array),np.std(array)]

[0.4076796068834272, 0.007111018085959277]

### 2. Now, let's improve the function by reducing the MC variations.
1. Use antithetic method: If `U` is uniform random variable, so is `1-U`
2. Also shift the RNs to match the mean, `1/def_rate`

In [11]:
# For example, antithetic method mean
n_sample = 10000
U = np.random.uniform(size=n_sample)
default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))

# Mean-matching means
default_time += 1/def_rate-default_time.mean()
(default_time.mean(), 1/def_rate)

(9.999999999999998, 10.0)

In [132]:
# No include the two new features: `antithetic` and `mean_match`

def corp_bond_cv(mat=10, def_rate=0.1, rf_rate=0.03, recovery=0.3, n_sample=1000, antithetic=True, mean_match=True):
    
    U = np.random.uniform(size=n_sample)
    
    if(antithetic):
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
        
    if(mean_match):
        default_time = -(1/def_rate)*np.log(U)
        default_time += 1/def_rate-default_time.mean()
    
    if(antithetic and mean_match):
        default_time = -(1/def_rate)*np.log(np.concatenate((U,1-U),axis=0))
        default_time += 1/def_rate-default_time.mean()

    else:
        default_time = -(1/def_rate)*np.log(U)
        
    price=np.where(default_time>mat,np.exp(-rf_rate*mat),recovery*np.exp(-rf_rate*default_time))
    
    return np.mean(price)

corp_bond_cv()

0.4408196218515868

In [133]:
# Find the mean and std by calling the function 100 times for (i) antithetic (ii) mean_match and (iii) both

antithetic=[ corp_bond_cv(antithetic=True, mean_match=False) for k in range(100)]
print("antithetic:",np.mean(antithetic),np.std(antithetic))

meanmatch=[ corp_bond_cv(antithetic=False, mean_match=True) for k in range(100)]
print("mean_matching:",np.mean(meanmatch),np.std(meanmatch))

both=[ corp_bond_cv() for k in range(100)]
print("both:",np.mean(both),np.std(both))

antithetic: 0.43941288296870695 0.00810610103819869
mean_matching: 0.44180129356723397 0.008080589999716893
both: 0.43999791880251693 0.0030017690826428466


### 3. Finally, what is the analytic value of the corporate bond? How does it compare to your MC result above?

In [120]:
def_rate = 0.1
rf_rate = 0.03
recovery = 0.3
mat = 10

def_rate*recovery*(1/(def_rate+rf_rate))*(1-np.exp(-(def_rate+rf_rate)*mat)) +np.exp(-(def_rate+rf_rate)*mat)

0.44040907156462505

In [ ]:
#The Monte Carlo simulation results obtained in part 2 are close to the analytic value of the corporate bond. Furthermore, the simulation combining both antithetic and mean-matching variance reduction techniques has the smallest simulation error. By increasing the number of simulations one should be able to arrive to a result closer to the analytic value of the bond.  